In [16]:
from transformers import pipeline, AutoTokenizer
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import os
from utils import *
config = load_config()


# os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"

LLAMA_PATH = "/home/htran/generation/biomed_instruct/models/llama_7b_lora/checkpoint-970"
LLAMA2_PATH = "/home/htran/generation/biomed_instruct/models/llama_2_7b_all_instructions/checkpoint-975"
IE_LLAMA_PATH = '/home/htran/generation/biomed_instruct/models/llama_2_7b_IE_updated_category/checkpoint-330'


# pipe = pipeline("text-generation", LLAMA2_PATH, max_new_tokens=128, device_map="auto")
# hf = HuggingFacePipeline(pipeline=pipe)

In [17]:
template = config.template('instructiontune')
instructions = config.template('instructions')

print("there a ~ e styles of instructions.\nThe instruction tells the model to extract important terms.\n")
print(instructions)

there a ~ e styles of instructions.
The instruction tells the model to extract important terms.

{'a': 'Extract and list the main health concerns or conditions mentioned in the medical note, making sure the terms are patient-friendly and easily understandable.', 'b': "Provide a clear summary of any treatment plans or medications prescribed in the note, including their purposes and how they should be administered, in layman's terms.", 'c': 'Identify and explain any recommended follow-up actions or appointments, ensuring that the instructions are straightforward and actionable for the patient.', 'd': "Interpret and summarize any test results or diagnostics from the note, using simple language to convey what these results mean for the patient's health.", 'e': 'Extract any preventative measures or lifestyle recommendations given in the medical note, presenting them in an easy-to-follow format for the patient.', 'hieu': "Translate the medical jargon in the provided sentence into layman's te

## ================================================== Load dataset
- load dataset : filtered notes, annotation info table


In [18]:
import pandas as pd

annotation_info_table = pd.read_pickle("../data/processed/annotation_info_table.pkl")
filtered_notes = pd.read_pickle("../data/processed/filtered_notes.pkl")

In [19]:
def get_note(noteid) :
    text = filtered_notes[filtered_notes['noteid'] == noteid]['text']
    annotations = annotation_info_table[annotation_info_table['noteid'] == noteid]['concept']
    annotations = annotations.tolist()
    return text, annotations

In [20]:
note_sample = filtered_notes.loc[2]['text']
# print(note_sample)

filtered_notes.head()

,category,noteid,text
2,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr..."
9,liver_failure,liver_failure.report70734.txt,"His diabetes has been under control, recently ..."
10,liver_failure,liver_failure.report72683.txt,A. Status post resection on XXXXX for T3 N2 M0...
13,liver_failure,liver_failure.report84657.txt,This is a followup infectious disease clinic f...
15,liver_failure,liver_failure.report88311.txt,1. Erosive seropositive rheumatoid arthritis t...


In [21]:
filtered_notes.shape

(51, 3)

In [22]:
annotation_info_table.head()

,note,concept,A,B,C,noteid
0,cancer.report20,non-Hodgkin's lymphoma,1.1,NaN,NaN,cancer.report20.txt
1,cancer.report20,non-Hodgkin's large B-cell lymphoma,1.1,NaN,NaN,cancer.report20.txt
2,cancer.report20,vincristine,2,NaN,NaN,cancer.report20.txt
3,cancer.report20,cyclophosphamide,2,NaN,NaN,cancer.report20.txt
4,cancer.report20,folinic acid,2,NaN,NaN,cancer.report20.txt


In [23]:
annotation_info_table['concept_modified'] = annotation_info_table.concept.apply(lambda x : str(x) + ", ")
annotations = annotation_info_table.groupby('noteid')['concept_modified'].sum()

In [24]:
# merge the notes
merged_notes = filtered_notes.merge(annotations, on='noteid')
merged_notes['concept_modified'] = merged_notes['concept_modified'].str.strip()
merged_notes.head(15)


,category,noteid,text,concept_modified
0,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr...","generalized OA, Diffuse OA, Gabapentin, naproxen,"
1,liver_failure,liver_failure.report70734.txt,"His diabetes has been under control, recently ...","diabetes, metformin, Lantus, Humalog, A1c, per..."
2,liver_failure,liver_failure.report72683.txt,A. Status post resection on XXXXX for T3 N2 M0...,"Polycythemia, peripheral vascular, colon cance..."
3,liver_failure,liver_failure.report84657.txt,This is a followup infectious disease clinic f...,"osteodiscitis, epidural phlegmon/abscess, MSSA..."
4,liver_failure,liver_failure.report88311.txt,1. Erosive seropositive rheumatoid arthritis t...,"rheumatoid arthritis, Ankylosing spondylitis, ..."
5,liver_failure,liver_failure.report100741.txt,"I had the pleasure of seeing your patient, Mrs...","carpal tunnel syndrome, Tramadol, hypercholest..."
6,liver_failure,liver_failure.report113518.txt,Mr. name returns for followup of rheumatoid ar...,"rheumatoid arthritis, Osteoarthritis, Humira i..."
7,liver_failure,liver_failure.report114863.txt,I had the pleasure of seeing Mr. name name in ...,"deformed pylorus and duodenal region, Intussus..."
8,liver_failure,liver_failure.report119233.txt,"I had the pleasure of seeing your patient, nam...","end-stage liver disease, alcoholic cirrhosis, ..."
9,liver_failure,liver_failure.report120088.txt,The patient is an 18-year-old woman with a his...,"cholecystitis, cholelithiasis, fatty liver, ab..."


In [25]:
NOTEID = "cancer.report20.txt"

note, concepts = get_note(NOTEID)

In [26]:
# lets build a dataset 
from datasets import Dataset

dataset = Dataset.from_pandas(merged_notes)
dataset


Dataset({
    features: ['category', 'noteid', 'text', 'concept_modified'],
    num_rows: 51
})

## ============================================ Zero Shot Testing

In [27]:
import torch
torch.cuda.device_count()

8

In [41]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn

# first test llama & llama2
# model = pipeline("text-generation", LLAMA2_PATH, max_new_tokens=128, device_map="auto")
# os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = "0,1,2,3"


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.66s/it]
/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization a

In [ ]:

DEFAULT_PAD_TOKEN = "<pad>"
DEFAULT_EOS_TOKEN= "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
max_target_length = 2048

model = AutoModelForCausalLM.from_pretrained(LLAMA2_PATH, cache_dir=None)
model.cuda()


tokenizer = AutoTokenizer.from_pretrained(LLAMA2_PATH, cache_dir=None)
tokenizer.add_special_tokens({
    'eos_token' : DEFAULT_EOS_TOKEN,
    'bos_token' : DEFAULT_BOS_TOKEN,
    'unk_token' : DEFAULT_UNK_TOKEN,
})

tokenizer.pad_token = tokenizer.eos_token

config = load_config()
instruction = config.template('instructions')
i1 = instruction['a']
i2 = instruction['hieu']

zeroshot_template = config.template('zeroshot')

In [42]:
zeroshot_template

'Instruction :  {instruction}\n\nContext :  {context}\n\nResponse : \n'

In [43]:
def process_texts(samples) :

    texts = samples['text']
    formated_texts = []
    for text in texts :
        new_text = zeroshot_template.format(
            instruction = i1,
            context = text
        )
        formated_texts.append(new_text)
    
    return {"questions" : formated_texts}

processed_dataset = dataset.map(process_texts, batched=True)

Map: 100%|██████████| 51/51 [00:00<00:00, 8353.23 examples/s]


In [44]:
list(map(lambda x : len(x), processed_dataset['questions']))

[4564,
 6234,
 5430,
 5521,
 4744,
 5926,
 4071,
 6254,
 5266,
 5412,
 6461,
 5290,
 5328,
 6162,
 3023,
 5556,
 4698,
 5486,
 6255,
 4233,
 5770,
 5924,
 3654,
 3970,
 5922,
 3576,
 3766,
 6457,
 6107,
 6469,
 4426,
 5019,
 5141,
 4704,
 6037,
 4590,
 5688,
 5249,
 5158,
 5983,
 5800,
 3026,
 4573,
 5212,
 4494,
 4748,
 5306,
 6309,
 5903,
 5264,
 6254]

In [45]:
# tokenize the texts
tokenized_texts = tokenizer(processed_dataset['questions'], 
                            return_tensors='pt', 
                            max_length=4000,
                            truncation='only_first',
                            padding=True)

In [46]:
import torch
with torch.no_grad() :
    output = model.generate(**tokenized_texts, max_new_tokens = 500)


/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.02 GiB. GPU 0 has a total capacty of 39.42 GiB of which 425.06 MiB is free. Process 3674365 has 31.26 GiB memory in use. Process 2573385 has 1.18 GiB memory in use. Including non-PyTorch memory, this process has 6.57 GiB memory in use. Of the allocated memory 5.63 GiB is allocated by PyTorch, and 11.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [62]:
# now decode the outputs
start_of_generate_index = tokenized_texts.input_ids.shape[1]
pred_output = tokenizer.batch_decode(output[:, start_of_generate_index:], skip_special_tokens=True)

In [63]:
pred_output

['1. generalized OA 2. D',
 'Љ1. Diabetes mellitus',
 'Љ Status post resection on T3 N2',
 'Љ4-L5 osteodiscitis',
 '. Erosive seropositive rhe',
 'Љ Hand weakness \nЉ Frequent falls',
 '. erosive rheumatoid ar',
 'Љаптоманија, пеп',
 'Љ Alcoholic cirrhosis ',
 'ЉРђРЅР°Р№',
 'Љ1. Osteoarthritis ',
 'ЉћЂЉћЂЉћЂЉ',
 '\nContext :  F/u on CAD',
 'Љиврплт: \n',
 '. Depression \n. Ankle pain',
 'Љ Coronary artery disease, status post',
 'Љ Diabetes: Glyburide ',
 'Љајнт: 74-',
 'Љ Coronary artery disease \nЉ',
 'Љ Methotrexate/sul',
 'Љ Hypertension \nЉ Alzheimer',
 '. \n. \n. \n.',
 'Љ Idiopathic pulmonary hemos',
 '. \n. \n. \n.',
 'Љ Hypertension, complicated with possible neph',
 '1. Lymphoma 2.',
 '. malignant large B cell diffuse l',
 'Љ\n',
 'Љ Methotrexate 17',
 '1. Rheumatoid arthrit',
 'Љивни болест\n\nContext',
 'Љ Graft: Well engrafted, off',
 'Љ78-year-old male with a',
 'Љејѕѕеd ',
 'Љамфом, степен III,',
 'ЉРђРЅР°Рј',
 'Љ Hypertension \nЉ Atrial f',
 '\\\\n1. Diabetes Mellit',
 

## ============================================ Few Shot Tuning

In [64]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn

# first test llama & llama2
# model = pipeline("text-generation", LLAMA2_PATH, max_new_tokens=128, device_map="auto")
# os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3"

DEFAULT_PAD_TOKEN = "<pad>"
DEFAULT_EOS_TOKEN= "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
max_target_length = 4096

model = AutoModelForCausalLM.from_pretrained(LLAMA2_PATH, cache_dir=None)

tokenizer = AutoTokenizer.from_pretrained(LLAMA2_PATH, cache_dir=None)
tokenizer.add_special_tokens({
    'eos_token' : DEFAULT_EOS_TOKEN,
    'bos_token' : DEFAULT_BOS_TOKEN,
    'unk_token' : DEFAULT_UNK_TOKEN,
})

tokenizer.pad_token = tokenizer.eos_token

config = load_config()
instruction = config.template('instructions')
i1 = instruction['a']
i2 = instruction['hieu']

fewshot_template = config.template('fewshot')

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.55s/it]
/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/wjang/miniconda3/envs/mna/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization a

In [65]:
fewshot_template

"Instruction :  {instruction}\n\nContext :  F/u on Osteoarthritis, chronic pain, HTN, Depression  The pt has generalized OA, in his shoulders, ankle and knees, the pain is about XXX in severity, worse with cold weather and activity, was on narocitcs in the past, then he stopped and went to ETOH for about 1.5 yrs, he went through detox, has not drank in the last month. He is currnetly on naproxen with no effect, was never tried on neuromodulators.  The pt also has HTN with hx of Afib that was single episode during an episode of pancreatitis, No CP, No SOB, No PNDs, No Orthopnea, No Edema, No Dizziness/Syncope, No Palpitations.  The pt also has depression, on wellbutrin, he's been on it for a couple of yrs with good results, no hx of suicidal or homocidal thoughts, No Depression, No Anxiety, Positive Sleep disturbance, No Sexual dysfunction.  The pt also has hx of Hep C, diagnosed in XXX, was treated in XXX, and since then he's been doing very well, he also had recurrent episodes of panc

In [66]:
def process_texts(samples) :

    example = config.template("fewshot_example")
    texts = samples['text']
    formated_texts = []
    for text in texts :
        contexts = example.format(instruction = i1)
        new_text = fewshot_template.format(
            instruction = i1,
            context = text
        )
        formated_texts.append(new_text)
    
    return {"contexts": [contexts]*len(formated_texts), "questions" : formated_texts}


processed_dataset = dataset.map(process_texts, batched=True)

Map: 100%|██████████| 51/51 [00:00<00:00, 1724.91 examples/s]


In [67]:
list(map(lambda x : len(x), processed_dataset['questions']))

[7077,
 8747,
 7943,
 8034,
 7257,
 8439,
 6584,
 8767,
 7779,
 7925,
 8974,
 7803,
 7841,
 8675,
 5536,
 8069,
 7211,
 7999,
 8768,
 6746,
 8283,
 8437,
 6167,
 6483,
 8435,
 6089,
 6279,
 8970,
 8620,
 8982,
 6939,
 7532,
 7654,
 7217,
 8550,
 7103,
 8201,
 7762,
 7671,
 8496,
 8313,
 5539,
 7086,
 7725,
 7007,
 7261,
 7819,
 8822,
 8416,
 7777,
 8767]

In [69]:
# tokenize the texts
tokenized_texts = tokenizer(processed_dataset['contexts'], 
                            processed_dataset['questions'], 
                            return_tensors='pt', 
                            padding=True,
                            max_length=4000,
                            truncation='only_first').to('cpu')

In [70]:
import torch
with torch.no_grad() :
    output = model.generate(**tokenized_texts, max_new_tokens = 500)

In [ ]:
# now decode the outputs
start_of_generate_index = tokenized_texts.input_ids.shape[1]
pred_output = tokenizer.batch_decode(output[:, start_of_generate_index:], skip_special_tokens=True)

In [ ]:
pred_output

['\nInstruction :  Extract and list the',
 '\nInstruction : \nExtract and list',
 '. \n. \n. \n.',
 '. L4-L5 osteodisc',
 '. Erosive seropositive rhe',
 '. "Instruction": "Extract and list',
 '. "Instruction": "Extract and list',
 '[{"type": "question", "question":',
 '. "Alcoholic cirrhosis',
 '. "Patient-friendly and easily understand',
 '. \n. \n. \n.',
 '. "Instruction": "Extract and list',
 '\nInstruction :  Extract and list the',
 '. "Patient": "Mr. name",',
 '. Esophagitis, presented with GI',
 '. \n. \n. \n.',
 '.Diabetes: I strongly emphasized on',
 '. "Patient_ID": "12',
 '. Coronary artery disease: Overall',
 '. \n. \n. \n.',
 'at 2021-02-',
 '. "Patient_Name": "Ms',
 ', "Idiopathic pulmonary hem',
 '. Extract and list the main health concerns or',
 '. Hypertension, complicated with possible neph',
 '\nInstruction :  Extract and list the',
 '. Left back and left-sided pain\n',
 '.000000000',
 '. \n. \n. \n.',
 '{"status":"success","data":{"id":1',
 '. \n. \n. \n.',
 '. "Graf